In [52]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from scipy import stats

In [53]:
df = pd.read_csv("train.csv")
df["Type_of_Cab"] = df["Type_of_Cab"].fillna("UNKNOWN")
df["Confidence_Life_Style_Index"] = df["Confidence_Life_Style_Index"].fillna("UNKNOWN")
month_mean = np.mean(df["Customer_Since_Months"])
df["Customer_Since_Months"] = df["Customer_Since_Months"].fillna(month_mean)
life_style_mean = np.mean(df["Life_Style_Index"])
df["Life_Style_Index"] = df["Life_Style_Index"].fillna(life_style_mean)
var1_mean = np.mean(df["Var1"])
df["Var1"] = df["Var1"].fillna(var1_mean)
y = np.array(df["Surge_Pricing_Type"])
X = df[["Trip_Distance", "Customer_Since_Months", "Life_Style_Index", "Customer_Rating", "Cancellation_Last_1Month", "Var1", "Var2", "Var3"]]
cab_encoder = OneHotEncoder(dtype=np.uint8)
cabs =  cab_encoder.fit_transform(df["Type_of_Cab"].values.reshape(-1,1)).todense()
lifestyle_encoder = OneHotEncoder(dtype=np.uint8)
life = lifestyle_encoder.fit_transform(df["Confidence_Life_Style_Index"].values.reshape(-1,1)).todense()
dest_encoder = OneHotEncoder(dtype=np.uint8)
dest = dest_encoder.fit_transform(df["Destination_Type"].values.reshape(-1,1)).todense()
gender_encoder = OneHotEncoder(dtype=np.uint8)
gender = gender_encoder.fit_transform(df["Gender"].values.reshape(-1,1)).todense()
X = np.array(np.hstack((X,cabs,life,dest,gender)))
print np.array(X),y

[[ 6.77      1.        2.42769  ...  0.        1.        0.      ]
 [29.47     10.        2.78245  ...  0.        0.        1.      ]
 [41.58     10.        2.802064 ...  0.        0.        1.      ]
 ...
 [40.17     10.        2.99565  ...  0.        1.        0.      ]
 [46.88      4.        3.04744  ...  0.        0.        1.      ]
 [31.96      7.        2.93773  ...  0.        0.        1.      ]] [2 2 2 ... 2 2 1]


In [54]:
models = []
fold = StratifiedKFold(n_splits=5)
for train_index, test_index in fold.split(X,y):
    X_train,X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #model = CatBoostClassifier(iterations=2000,learning_rate=0.015, verbose=0)
    model = CatBoostClassifier(verbose=0)
    model.fit(X_train,y_train)
    print model.score(X_test, y_test)
    models.append(model)

0.7042494208787453
0.707667185660578
0.704097520221775
0.7089472884703023
0.7021001860924385


In [31]:
for model in models:
    print model.learning_rate_

0.02999999933
0.02999999933
0.02999999933
0.02999999933
0.02999999933


In [55]:
df = pd.read_csv("test.csv")
df["Type_of_Cab"] = df["Type_of_Cab"].fillna("UNKNOWN")
df["Confidence_Life_Style_Index"] = df["Confidence_Life_Style_Index"].fillna("UNKNOWN")
df["Customer_Since_Months"] = df["Customer_Since_Months"].fillna(month_mean)
df["Life_Style_Index"] = df["Life_Style_Index"].fillna(life_style_mean)
df["Var1"] = df["Var1"].fillna(var1_mean)
X = df[["Trip_Distance", "Customer_Since_Months", "Life_Style_Index", "Customer_Rating", "Cancellation_Last_1Month", "Var1", "Var2", "Var3"]]
cabs =  cab_encoder.transform(df["Type_of_Cab"].values.reshape(-1,1)).todense()
life = lifestyle_encoder.transform(df["Confidence_Life_Style_Index"].values.reshape(-1,1)).todense()
dest = dest_encoder.transform(df["Destination_Type"].values.reshape(-1,1)).todense()
gender = gender_encoder.transform(df["Gender"].values.reshape(-1,1)).todense()
X = np.array(np.hstack((X,cabs,life,dest,gender)))

In [56]:
y = []
for model in models:
    y.append(model.predict(X))
y = np.array(y)
print y.shape
#y = stats.mode(y,axis=0).mode.reshape(-1,)
(values,counts) = np.unique(y,return_counts=True)
ind=np.argmax(counts)
y = y[ind].reshape(-1,)
# print y

(5, 87395, 1)


In [57]:
ids = df["Trip_ID"].values
with open("sub.csv","w") as file:
    csv_writer = csv.writer(file)
    csv_writer.writerow(["Trip_ID", "Surge_Pricing_Type"])
    i = 0
    for id in ids:
        csv_writer.writerow([id,int(y[i])])
        i += 1